In [1]:
import pandas as pd
import string
import os

In [2]:
def stemmer(word):
    first_3 =  ['عال', 'فال','بال','وال']
    first_2 =  ['مي', 'مت','ال']
    first_1 = ['ا', 'ي', 'ت',  'ن','م']
    last_1 = ['و', 'ك','ا']
    last_2 = ['نا', 'ني', 'او', 'لو', 'لي', 'لك', 'تو', 'اك''كم', 'يو','هم','وه','يه']
    last_3 = ['وهم','وكم','لنا', 'لكم', 'لهم','يهم','فال']

    removed = False
    # case of negative word (example: 'مايجيكمش')
    if word.startswith('ما') and word.startswith('ش') and len(word)>4 :
        word = word[2:len(word)-1]
    for prefix in first_3:
        if word.startswith(prefix) and len(word)>3:
            word = word[3:]
            removed = True
            break
    for prefix in first_2:
        if word.startswith(prefix) and len(word)>3:
            word = word[2:]
            removed = True
            break
    if not(removed):
        for prefix in first_1:
            if word.startswith(prefix) and len(word)>4:
                word = word[1:]
                break
    removed = False
    for prefix in last_3:
        if word.endswith(prefix) and len(word)>4:
            word = word[0:len(word)-3]
            removed = True
            break
    if not(removed):
        for prefix in last_2:
            if word.endswith(prefix) and len(word)>3:
                word = word[0:len(word)-2]
                removed = True
                break
    if not(removed):
        for prefix in last_1:
            if word.endswith(prefix) and len(word)>2:
                word = word[0:len(word)-1]
                break
    return word
def remove_punctuation(text):
    # Get the string of all punctuation characters
    punctuation = string.punctuation
    punctuation += '؟،'

    # Remove punctuation characters from the text
    text_without_punctuation = ''.join(char for char in text if char not in punctuation)
    return text_without_punctuation

In [3]:
def is_arabic(word):
    arabic_letters = []
    start = ord('\u0600')  # Start of Arabic Unicode range
    end = ord('\u06FF')  # End of Arabic Unicode range

    # Iterate through the Unicode range and append Arabic letters to the array
    for codepoint in range(start, end + 1):
        character = chr(codepoint)
        arabic_letters.append(character)
    for i in word:
        if not(i in arabic_letters):
            return False
    return True


def exist(file_path, word):
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            if (stemmer(word) in line.strip('\n')):
                return True
    return False


In [4]:
file_path = r'C:\Users\Admin\Desktop\dict.txt'
i = 0
datasets_path = r'C:\Users\Admin\Desktop\datasets'
datasets = [os.path.join(datasets_path, file) for file in os.listdir(datasets_path) if file.endswith('.csv')]
for dataset in datasets:
    df = pd.read_csv(dataset)
    first_column_name = df.columns[0]
    first_column = df[first_column_name]
    # Open the file in append mode
    with open(file_path, 'a', encoding='utf-8') as file:
        added_words = []
        for post in first_column:
            # Check if the post is not NaN
            if pd.notna(post):
                post = remove_punctuation(post)
                words = post.split(' ')
                for word in words:
                    if is_arabic(word):
                        stemmed_word = stemmer(word)
                        if not (exist(file_path,stemmed_word) or (stemmed_word in added_words)):
                            added_words.append(stemmed_word)
                            file.write(stemmed_word + '\n')
                            i += 1

print(f'Added {i} words.')

Added 4685 words.
